In [ ]:
# Importing Python and external packages
import os
import sys
import json
import importlib
import pandas as pd
import numpy as np
import sklearn as sk
import scipy
import matplotlib.pyplot as plt

from itertools import product

In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)

In [ ]:
import retap_utils.utils_dataManagement as utils_dataMn


In [ ]:
# DEFINE RETAP OUTCOME DIRECTORY
ft_path = os.path.join(utils_dataMn.get_local_proj_dir(),
                       'aDBS_tapping',
                       'retap_results', 'features')
assert os.path.exists(ft_path), 'defined ft_path does not exist'
fig_path = os.path.join(utils_dataMn.get_local_proj_dir(),
                       'aDBS_tapping', 'figures')
if not os.path.exists(fig_path): os.makedirs(fig_path)

In [ ]:
# DEFINE FEATURES OF INTEREST
FEAT_SEL = ['trace_RMSn',
            'coefVar_impactRMS',
            'coefVar_intraTapInt',
            'mean_raise_velocity']

explore aDBS data content


In [ ]:
subs = np.unique([f.split('_')[1] for f in os.listdir(ft_path)])

print(len(subs), subs)

In [ ]:
files_dict = {}
for sub in subs:
    print(sub)
    files = [f for f in os.listdir(ft_path) if f'{sub}_' in f and 'medoff' in f.lower()]
    on_file = [f for f in files if 'first' in f and 'cdbs' in f.lower()
                and 'stimon' in f.lower()][0]
    off_file = [f for f in files if 'first' in f and 'cdbs' in f.lower()
                and 'stimoff' in f.lower()][0]
    
    with open(os.path.join(ft_path, off_file), 'r') as f:
        off_file = json.load(f)
    with open(os.path.join(ft_path, on_file), 'r') as f:
        on_file = json.load(f)

    files_dict[sub] = {'on': on_file, 'off': off_file}

In [ ]:
box_lists = []
box_labels = []

for ft, con in product(FEAT_SEL, ['off', 'on']):
    box_labels.append(f'{ft} {con}')
    values = [files_dict[s][con][ft] for s in subs]
    box_lists.append(values)

In [ ]:
method = 'norm'

for i_ft, ft in enumerate(FEAT_SEL):

    off = box_lists[i_ft * 2]
    on = box_lists[i_ft * 2 + 1]

    if method == 'norm':
        norm_value = np.nanmax(off)
        off = np.array(off) / norm_value
        on = np.array(on) / norm_value
    elif method == 'std':
        norm_m = np.nanmean(off)
        norm_sd = np.nanstd(off)
        off = (np.array(off) - norm_m) / norm_sd
        on = (np.array(on) - norm_m) / norm_sd

    box_lists[i_ft * 2] = off
    box_lists[i_ft * 2 + 1] = on

In [ ]:
SCATTER = True
SCATTER_LINES = True

fig_name = f'firstRun_4feats_cDBSOffvsOn_{method}'

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

fontsize=14

ax.boxplot(box_lists, positions=np.arange(len(box_lists)),)
ax.set_xticklabels(box_labels, rotation=90,
                   size=fontsize)
if method == 'norm':
    ylabel = 'Normalised feature values\n(against max OFF value)'
elif method == 'std':
    ylabel = 'Standardised feature values\n(against OFF values)'

if SCATTER:
    for i_x, values in enumerate(box_lists):
        ax.scatter([i_x] * len(values), values,
                   alpha=.3)
if SCATTER_LINES:
    for i_ft in np.arange(len(FEAT_SEL)):
        for i_sub in np.arange(len(box_lists[i_ft])):
            y1 = box_lists[i_ft * 2][i_sub]
            y2 = box_lists[i_ft * 2 + 1][i_sub]
            ax.plot([i_ft*2, i_ft*2+1], [y1, y2],
                    c='gray', alpha=.3)

ax.set_ylabel(ylabel,
              size=fontsize)

plt.tick_params(axis='both', labelsize=fontsize, size=fontsize)

plt.tight_layout()

plt.savefig(os.path.join(fig_path, fig_name), dpi=300,
            facecolor='w',)
plt.close()